In [1]:
import mlflow
from mlflow.models import infer_signature

import ultralytics
import time
import cv2
import torch
import numpy as np
import os
import onnx

from ultralytics import YOLO

In [18]:
os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"

# # Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# # Create a new MLflow Experiment
mlflow.set_experiment("YOLO Models")

# Create a new YOLO model from scratch
# model = YOLO("yolo11n.yaml")

model_name = 'yolov5m'

# Load a pretrained YOLO model (recommended for training)
model = YOLO(f"{model_name}.pt")
# yolo_model = torch.hub.load(f'ultralytics/yolov6', model_name, pretrained=True)
model = model.export(format='onnx')

onnx_model = onnx.load(model)

mlflow_model = mlflow.onnx.log_model(onnx_model, artifact_path=model_name, registered_model_name=model_name)

PRO TIP 💡 Replace 'model=yolov5m.pt' with new 'model=yolov5mu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

Ultralytics 8.3.21 🚀 Python-3.10.12 torch-2.5.0+cu124 CPU (13th Gen Intel Core(TM) i7-13700HX)
YOLOv5m summary (fused): 248 layers, 25,091,536 parameters, 0 gradients, 64.2 GFLOPs

PyTorch: starting from 'yolov5mu.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (48.2 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.36...
ONNX: export success ✅ 1.0s, saved as 'yolov5mu.onnx' (96.0 MB)

Export complete (1.4s)
Results saved to /home/darkaengl/Project/REAL/scripts/mlops
Predict:         yolo predict task=detect model=yolov5mu.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov5mu.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


2024/11/05 11:31:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'yolov5m' already exists. Creating a new version of this model...
2024/11/05 11:31:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: yolov5m, version 2
Created version '2' of model 'yolov5m'.


In [37]:
from PIL import Image

img = '/home/darkaengl/Project/REAL/scripts/mlops/coco8/images/train/000000000025.jpg'
img = Image.open(img).resize([640, 640])
img = np.asarray(img).transpose(2,1,0)
# img = np.ascontiguousarray(img)
img.shape

(3, 640, 640)

In [38]:
type(img)

numpy.ndarray

In [41]:
from mlserver.codecs import NumpyRequestCodec
from mlserver.codecs import NumpyCodec
from mlserver.types import InferenceRequest
import requests

In [42]:
inference_request = InferenceRequest(
    inputs = [
        NumpyCodec.encode_input("image", img)
    ]
)

In [43]:
# inference_request.model_dump()

In [44]:
import requests
# endpoint = "http://localhost:8080/invocations"
# response = requests.post(endpoint, json=inference_request)
# response = requests.post(endpoint, files={"file": img})
r = requests.post('http://0.0.0.0:8080/v2/models/yolo_models/infer', json=inference_request.model_dump())
r.json()['outputs'][0]['data'][0] =='false'

True

In [41]:
# onnx.checker.check_model(onnx_model)

In [1]:
# import torch
# import torch.nn as nn

# class YOLOWithFFNN(nn.Module):
#     def __init__(self, yolo_model, num_ffnn_outputs=10, freeze_yolo=True):
#         super(YOLOWithFFNN, self).__init__()

#         # Load the pretrained YOLOv5 model
#         yolov5_model = yolo_model
#         yolov5_model.eval()  # Set YOLO to evaluation mode

#         model_structure = yolov5_model.model.model

#         # Optionally freeze YOLO parameters
#         if freeze_yolo:
#             for param in yolov5_model.parameters():
#                 param.requires_grad = False

#         # Use all layers except the last detection layer (if extracting features)
#         self.yolo_feature_extractor = nn.Sequential(*list(model_structure.children())[:-1])  # Up to last layer

#         # Define the feed-forward neural network
#         # Adjust input dimension based on YOLO's feature output shape
#         self.flatten = nn.Flatten()
#         self.ffnn = nn.Sequential(
#             nn.Linear(512 * 20 * 20, 1024),  # Replace 512, 20, 20 with appropriate dimensions
#             nn.ReLU(),
#             nn.Linear(1024, 512),
#             nn.ReLU(),
#             nn.Linear(512, num_ffnn_outputs)  # Final output layer
#         )

#     def forward(self, x):
#         # Forward pass through YOLO feature extractor
#         yolo_features = self.yolo_feature_extractor(x)
        
#         # Flatten YOLO features and pass through FFNN
#         yolo_features = self.flatten(yolo_features)
#         output = self.ffnn(yolo_features)
#         return output

# # Instantiate the model with a YOLOv5 model file
# yolo_ffnn_model = YOLOWithFFNN(yolo_model, num_ffnn_outputs=10)

# # Example input tensor
# input_tensor = torch.randn(1, 3, 640, 640)  # Adjust to match YOLO's input size
# output = yolo_ffnn_model(input_tensor)

# print(output.shape)  # Expected output shape: [batch_size, num_ffnn_outputs]
